In [1]:
! pip install -U bitsandbytes accelerate transformers peft trl --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 33.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 19.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, BitsAndBytesConfig, GenerationConfig
import bitsandbytes
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from tqdm import tqdm
from trl import SFTTrainer

## Import the data from Hugging face

In [3]:
# dataset = load_dataset('samhog/psychology-10k', split = ['train'])[0]
# dataset.save_to_disk('./dataset')

dataset = load_from_disk('./dataset')
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9846
})

let's walk trough the data

In [4]:
def print_conv(me, model):
    for i, j in zip(me, model):
        print(
            f"""
Me : {i}

Param mitr : {j}\n\n
            """
        )

In [5]:
print_conv(dataset['input'][:3], dataset['output'][:3])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : It's common to feel anxious at times, and there can be many reasons for it. Have there been any recent changes or stressors in your life that may be contributing to your anxiety? Let's work together to identify any triggers and develop coping strategies to manage your anxiety.


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : It's understandable to feel worried and suspicious in this situation. Have you talked to your partner about your concerns? It's important to communicate openly and honestly with them. If you're still feeling uncertain, we can work on developing a plan to address the situation in a healthy and constructive way.


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : It sounds like you're going through a difficult time. Let's work on creating a 

In [6]:
dataset.shape

(9846, 3)

In [7]:
Dataset = dataset.train_test_split(test_size = 0.2)
Dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 7876
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1970
    })
})

## Import our model with its tokenizer

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

In [31]:
# Importing model from the Hugging Face hub

# model_name = 'google/flan-t5-small'
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype = torch.bfloat16,
#     # quantization_config = bnb_config
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Saving the model

# model.save_pretrained('/kaggle/working/Model')
# tokenizer.save_pretrained('/kaggle/working/Model')

In [24]:
import gc
def clean():
    for i in range(15):
      gc.collect()
      torch.cuda.empty_cache()

In [9]:
# Importing model saved locally
tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Model', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
                                              device_map="auto",
                                              torch_dtype = torch.bfloat16,
                                              # quantization_config = bnb_config
                                             )

Let's try some stuff with tokenizer

In [10]:
tokenizer("hey", return_token_type_ids = False)

{'input_ids': [3, 13133, 1], 'attention_mask': [1, 1, 1]}

In [11]:
device = torch.device('cuda')
tokenizer.decode(
    model.generate(
        tokenizer(
            dataset['input'][0], return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device)
    )[0], skip_special_tokens= True
)


"I'm feeling really anxious lately and I don't know why."

In [16]:
def inference(input_data, param_mitr = model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent + task for sent in input_data]
    output = param_mitr.generate(
        tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device),
        generation_config = GenerationConfig(max_new_token = 200)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)



In [17]:
inference(dataset['input'][:4])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : I'm feeling a lot more anxious.


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : I should give them a call.


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : I'm feeling really overwhelmed with work and school. I don't know how to manage my


            

Me : I'm having trouble sleeping and I'm constantly tired. I think it might be because of my medication.

Param mitr : I'm having trouble sleeping and I'm constantly tired. I think it might be because of


            


The model is showing good result on ICL (In Context Learning) with zero shot inference.

In [18]:
def tokenize_function(example):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + question + '\n' + task for question in example['input']]
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['output'], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


tokenized_datasets = Dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input', 'output', 'instruction'])

Map:   0%|          | 0/7876 [00:00<?, ? examples/s]

Map:   0%|          | 0/1970 [00:00<?, ? examples/s]

## Fine tune our model


In [19]:
config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r = 32,
    target_modules = "all-linear",
    lora_alpha=32,
    lora_dropout=0.05
)

In [20]:
Model = get_peft_model(model, config)
Model.print_trainable_parameters()

trainable params: 5,111,808 || all params: 82,072,960 || trainable%: 6.2284


I have already saved the model and will not retrieve it from the hub

In [21]:
TrainArgs = TrainingArguments(
    output_dir = './Model',
    learning_rate = 1e-3,
    num_train_epochs = 2,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps=1,
    load_best_model_at_end = True,
    report_to = ['tensorboard']
)

In [22]:
trainer = Trainer(
    model = Model,
    args = TrainArgs,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [25]:
clean()
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.679700,1.635533
2,1.640600,1.617679


TrainOutput(global_step=3938, training_loss=1.8211338242762825, metrics={'train_runtime': 1269.2217, 'train_samples_per_second': 12.411, 'train_steps_per_second': 3.103, 'total_flos': 3175508355317760.0, 'train_loss': 1.8211338242762825, 'epoch': 2.0})

In [28]:
def peft_infrence(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ).to('cuda'),
        generation_config = GenerationConfig(max_new_tokens=100, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [29]:
peft_infrence(dataset['input'][-4:])


Me : I feel like I'm stuck in a rut and don't know how to move forward.

Param mitr : It's important to identify the root cause of your stuckness and develop strategies to move forward. Let's work together to identify the root cause of your stuckness and develop a plan to move forward.


            

Me : I'm having trouble connecting with my teenage daughter.

Param mitr : Parenting can be challenging, but it's important to communicate openly and honestly with your daughter. Let's work together to develop a plan to improve your relationship with your daughter.


            

Me : I'm struggling with addiction and don't know where to turn.

Param mitr : Addiction is a complex issue, but it's important to seek help. Let's explore your addiction and develop a treatment plan that works for you. We can also discuss treatment options such as therapy or medication.


            

Me : I'm having trouble making decisions and sticking to them.

Param mitr : Decision-making can be challengi

In [30]:
trainer.model.save_pretrained('./peft_for_param_mitr')